# 1. Pick 10 random labels from training set

In [23]:
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
import pickle
import random
import math
import numpy as np

In [2]:
import pickle
with open('/agh/projects/noelia/NLP/zymCTRL/dataset_preparation/ec_classes_freq-train.pkl', 'rb') as file:
    # A new file will be created
    ec_classes_freq_train = pickle.load(file)

In [3]:
import random
random_training_labels = random.choices(list(ec_classes_freq_train.keys()),k=10)

In [4]:
random_training_labels

['1.1.1.29',
 '1.14.12.22',
 '1.4.2.-',
 '2.4.1.7',
 '2.7.7.2',
 '4.1.3.27',
 '2.1.1.219',
 '2.7.1.108',
 '1.8.1.20',
 '1.14.13.2']

###  Load model and tokenizer

In [9]:
device = torch.device("cuda")    
tokenizer = AutoTokenizer.from_pretrained('/agh/projects/noelia/NLP/zymCTRL/dataset_preparation/tokenizer')
model = GPT2LMHeadModel.from_pretrained('/agh/projects/noelia/NLP/zymCTRL/model').to(device)
                                        
                                       

In [16]:
def calculatePerplexity(input_ids,model,tokenizer):
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)


In [22]:
ppls={}
for label in random_training_labels:
    input_ids = tokenizer.encode(label,return_tensors='pt').to(device)
    
    outputs = model.generate(
        input_ids,
        top_k=9,
        repetition_penalty=1.2,
        max_length=1024,
        eos_token_id=1,
        pad_token_id=0,
        do_sample=True,
        num_return_sequences=20)
    
    ppls[label] = [calculatePerplexity(output, model, tokenizer) for output in outputs ]
  


In [25]:
for label,ppl_values in ppls.items():
    print(label)
    print(np.mean(ppl_values))

1.1.1.29
3.6004226817688965
1.14.12.22
2.7306436020064
1.4.2.-
6.520056013567979
2.4.1.7
3.436233128206016
2.7.7.2
1.8069302782952466
4.1.3.27
1.9432983414756877
2.1.1.219
3.446483115689408
2.7.1.108
4.389680475286834
1.8.1.20
3.207328023816448
1.14.13.2
2.505419333032168
